Be sure to change runtime type to GPU if on Google Colab

In [1]:
import math
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import os

In [2]:
from google.colab import drive
drive.mount('content/')

Drive already mounted at content/; to attempt to forcibly remount, call drive.mount("content/", force_remount=True).


In [3]:
from keras.preprocessing import image
test_image = image.load_img('/content/content/MyDrive/AI & Deep Learning Project/Garbage classification/test/cardboard/cardboard102.jpg')
img_array = image.img_to_array(test_image)

In [4]:
img_array.shape

(384, 512, 3)

In [5]:
n_classes = len(os.listdir('/content/content/MyDrive/AI & Deep Learning Project/images'))
n_classes

6

In [6]:
img_array = image.img_to_array(test_image)
keras.applications.xception.preprocess_input(img_array).shape

(384, 512, 3)

In [7]:
TEST_PATH = '/content/content/MyDrive/AI & Deep Learning Project/Garbage classification/test'
TRAIN_PATH = '/content/content/MyDrive/AI & Deep Learning Project/Garbage classification/train'

X_test_list = []
y_test_list = []

X_train_list = []
y_train_list = []

for folder in os.listdir(TEST_PATH):
  FOLDER_PATH = os.path.join(TEST_PATH, folder)
  for im in os.listdir(FOLDER_PATH):
    img = image.load_img(os.path.join(TEST_PATH, folder, im), target_size=(224, 224))
    img_array = image.img_to_array(img)
    processed_img = keras.applications.xception.preprocess_input(img_array)
    X_test_list.append(list(processed_img))
    y_test_list += [folder] # label of the image being passed in
    if len(y_test_list)%50 == 0:
      print(len(y_test_list), 'images loaded into test')

for folder in os.listdir(TRAIN_PATH):
  FOLDER_PATH = os.path.join(TRAIN_PATH, folder)
  for im in os.listdir(FOLDER_PATH):
    img = image.load_img(os.path.join(TRAIN_PATH, folder, im), target_size=(224, 224))
    img_array = image.img_to_array(img)
    processed_img = keras.applications.xception.preprocess_input(img_array)
    X_train_list.append(list(processed_img))
    y_train_list += [folder] # label of the image being passed in
    if len(y_train_list)%50 == 0:
      print(len(y_train_list), 'images loaded into train')

50 images loaded into test
100 images loaded into test
150 images loaded into test
200 images loaded into test
250 images loaded into test
300 images loaded into test
350 images loaded into test
400 images loaded into test
450 images loaded into test
500 images loaded into test
50 images loaded into train
100 images loaded into train
150 images loaded into train
200 images loaded into train
250 images loaded into train
300 images loaded into train
350 images loaded into train
400 images loaded into train
450 images loaded into train
500 images loaded into train
550 images loaded into train
600 images loaded into train
650 images loaded into train
700 images loaded into train
750 images loaded into train
800 images loaded into train
850 images loaded into train
900 images loaded into train
950 images loaded into train
1000 images loaded into train
1050 images loaded into train
1100 images loaded into train
1150 images loaded into train
1200 images loaded into train
1250 images loaded in

In [8]:
X_test = np.array(X_test_list)
X_train_full = np.array(X_train_list)
y_test = pd.factorize(y_test_list)[0].reshape(-1, 1)
y_train_full = pd.factorize(y_train_list)[0].reshape(-1, 1)

In [9]:
test_shuffler = np.random.permutation(len(X_test))
X_test = X_test[test_shuffler]
y_test = y_test[test_shuffler]

train_full_shuffler = np.random.permutation(len(X_train_full))
X_train_full = X_train_full[train_full_shuffler]
y_train_full = y_train_full[train_full_shuffler]

In [10]:
valid_size = int(len(y_test)/2)
X_train, X_valid = X_train_full[:-valid_size], X_train_full[-valid_size:]
y_train, y_valid = y_train_full[:-valid_size], y_train_full[-valid_size:]

In [11]:
base_model = keras.applications.xception.Xception(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

In [12]:
base_model.trainable = False

model = keras.models.Sequential()
model.add(base_model)
model.add(keras.layers.GlobalAveragePooling2D())
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(6, activation='softmax')) # prediction layer

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 7, 7, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               524544    
_________________________________________________________________
batch_normalization_4 (Batch (None, 256)               1024      
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
flatten (Flatten)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 1

In [17]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
# implementing a custom callback to stop training if we reach accuracy >0.999 because that indicates overfitting on this relatively small dataset
class OverfittingCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if logs.get('accuracy') > 0.999:
      self.model.stop_training = True
      print('Trying to prevent overfitting - stopping training')

In [18]:
history = model.fit(X_train, y_train, epochs=40, validation_data = (X_test, y_test), callbacks = [keras.callbacks.EarlyStopping(patience=3, monitor='val_accuracy'), OverfittingCallback()])

Epoch 1/40
55/55 [==============================] - 43s 199ms/step - loss: 0.7740 - accuracy: 0.7526 - val_loss: 0.5388 - val_accuracy: 0.8185
Epoch 2/40
55/55 [==============================] - 9s 162ms/step - loss: 0.3128 - accuracy: 0.8886 - val_loss: 0.4914 - val_accuracy: 0.8147
Epoch 3/40
55/55 [==============================] - 9s 162ms/step - loss: 0.2018 - accuracy: 0.9229 - val_loss: 0.4173 - val_accuracy: 0.8571
Epoch 4/40
55/55 [==============================] - 9s 164ms/step - loss: 0.1290 - accuracy: 0.9594 - val_loss: 0.5018 - val_accuracy: 0.8359
Epoch 5/40
55/55 [==============================] - 9s 164ms/step - loss: 0.0827 - accuracy: 0.9811 - val_loss: 0.4516 - val_accuracy: 0.8591
Epoch 6/40
55/55 [==============================] - 9s 166ms/step - loss: 0.0590 - accuracy: 0.9863 - val_loss: 0.4772 - val_accuracy: 0.8514
Epoch 7/40
55/55 [==============================] - 9s 166ms/step - loss: 0.0496 - accuracy: 0.9886 - val_loss: 0.4723 - val_accuracy: 0.8591
Epoch

In [19]:
base_model.trainable = True
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 7, 7, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               524544    
_________________________________________________________________
batch_normalization_4 (Batch (None, 256)               1024      
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
flatten (Flatten)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 1

In [20]:
history = model.fit(X_train, y_train, epochs=40, validation_data=(X_test, y_test), callbacks=[keras.callbacks.EarlyStopping(patience=3, monitor='val_accuracy'), OverfittingCallback()])

Epoch 1/40
55/55 [==============================] - 9s 169ms/step - loss: 0.0215 - accuracy: 0.9971 - val_loss: 0.6023 - val_accuracy: 0.8436
Epoch 2/40
55/55 [==============================] - 9s 171ms/step - loss: 0.0186 - accuracy: 0.9960 - val_loss: 0.5574 - val_accuracy: 0.8533
Epoch 3/40
55/55 [==============================] - 9s 171ms/step - loss: 0.0098 - accuracy: 1.0000 - val_loss: 0.5097 - val_accuracy: 0.8745
Epoch 4/40
55/55 [==============================] - 9s 172ms/step - loss: 0.0070 - accuracy: 1.0000 - val_loss: 0.5248 - val_accuracy: 0.8629
Epoch 5/40
55/55 [==============================] - 10s 174ms/step - loss: 0.0051 - accuracy: 1.0000 - val_loss: 0.5418 - val_accuracy: 0.8610
Epoch 6/40
55/55 [==============================] - 9s 173ms/step - loss: 0.0057 - accuracy: 0.9994 - val_loss: 0.5480 - val_accuracy: 0.8668


In [33]:
incorrect = dict()
for idx, predictions in enumerate(model.predict(X_valid)):
  if np.argmax(predictions) != y_valid[idx][0]:
    incorrect[idx] = {'Predicted': np.argmax(predictions), 'Actual': y_valid[idx][0]}

incorrect_preds = pd.DataFrame.from_dict(incorrect, orient='index')

In [35]:
incorrect_preds.reset_index()

,index,Predicted,Actual
0,1,4,2
1,3,2,4
2,19,0,4
3,20,0,2
4,39,0,3
5,44,1,3
6,47,3,0
7,49,0,3
8,51,2,3
9,52,3,4
